In [1]:
import importlib

import pandas as pd
import numpy as np
from getpass import getpass
import geopandas as gpd
import geopy

importlib.import_module("helpers")

from helpers import convert_elapsed_time, extract_num, build_era
from helpers import listing_type, roof_description, create_dummy
#from helpers import get_coords

pd.options.display.max_columns = None
pd.options.display.max_rows = 100

In [41]:
df = pd.read_json("../scraping/ams_sale.json")

In [3]:
df.shape

(4091, 78)

In [4]:
df.head()

,address,postcode,city,OVE-Vraagprijs,OVE-Vraagprijs per m²,OVE-Aangeboden sinds,OVE-Status,OVE-Aanvaarding,BOU-Soort woonhuis,BOU-Soort bouw,BOU-Bouwjaar,BOU-Soort dak,OPP-,OPP-Perceel,OPP-Inhoud,IND-Aantal kamers,IND-Aantal badkamers,IND-Badkamervoorzieningen,IND-Aantal woonlagen,IND-Voorzieningen,ENE-Energielabel,ENE-Isolatie,ENE-Verwarming,ENE-Warm water,ENE-Cv-ketel,KAD-,BUI-Tuin,BUI-Balkon/dakterras,GAR-Soort garage,GAR-Capaciteit,GAR-Voorzieningen,PAR-Soort parkeergelegenheid,BUI-Ligging,BUI-Voortuin,BUI-Ligging tuin,OVE-Servicekosten,BOU-Specifiek,BUI-Zonneterras,BER-Schuur/berging,BER-Voorzieningen,BUI-Achtertuin,BER-Isolatie,BOU-Keurmerken,BUI-Patio/atrium,OVE-Bijdrage VvE,BOU-Soort appartement,BOU-Bouwperiode,IND-Gelegen op,VVE-Inschrijving KvK,VVE-Jaarlijkse vergadering,VVE-Periodieke bijdrage,VVE-Reservefonds aanwezig,VVE-Onderhoudsplan,VVE-Opstalverzekering,GAR-Isolatie,BOU-Toegankelijkheid,OVE-Oorspronkelijke vraagprijs,ENE-Voorlopig energielabel,OVE-Huurprijs,OVE-Huurovereenkomst,BUI-Plaats,BUI-Zijtuin,OVE-Oorspronkelijke huurprijs,BOU-Soort overig aanbod,BED-Praktijkruimte,OVE-Koopmengvorm,BOU-Soort parkeergelegenheid,IND-Capaciteit,IND-Afmetingen,VEI-Prijs,VEI-Veilingperiode,VEI-Soort veiling,VEI-Veilingpartij,BED-Bedrijfsruimte,BED-Kantoorruimte,BED-Winkelruimte,IND-Perceel,BOU-Soort object
0,Buiksloterbreek 41,1034 XC,Amsterdam,€ 475.000 kosten koper,€ 5.220,26 september 2020,Beschikbaar,In overleg,"Eengezinswoning, geschakelde woning",Bestaande bouw,1994,Plat dak,,51 m²,379 m³,4 kamers (3 slaapkamers),1 badkamer en 1 apart toilet,Douche en toilet,2 woonlagen,"Buitenzonwering, schuifpui en TV kabel",B,Volledig geïsoleerd,Cv-ketel,Cv-ketel,Intergas HRE (gas gestookt combiketel uit 2010...,,Zonneterras,Dakterras aanwezig,Inpandig,1 auto,Stromend water,"Betaald parkeren, openbaar parkeren en parkeer...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Arènpalmstraat 16,1104 DB,Amsterdam,€ 425.000 kosten koper,€ 3.400,17 september 2020,Beschikbaar,In overleg,"Eengezinswoning, hoekwoning",Bestaande bouw,2008,Plat dak,,98 m²,429 m³,6 kamers (4 slaapkamers),1 badkamer en 1 apart toilet,"Ligbad, douche en toilet",3 woonlagen,Mechanische ventilatie en TV kabel,A,Volledig geïsoleerd,Cv-ketel en gedeeltelijke vloerverwarming,Cv-ketel,Gas gestookt combiketel,,Voortuin,Dakterras aanwezig en balkon aanwezig,NaN,NaN,NaN,Op eigen terrein en openbaar parkeren,"Aan rustige weg, aan water, in woonwijk en vri...","36 m² (7 meter diep en 5,09 meter breed)",Gelegen op het zuidoosten,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Brigantijnkade 46,1086 VB,Amsterdam,€ 625.000 kosten koper,€ 4.340,18 september 2020,Beschikbaar,In overleg,"Villa, vrijstaande woning (waterwoning)",Bestaande bouw,2009,Plat dak bedekt met bitumineuze dakbedekking,,133 m²,519 m³,5 kamers (4 slaapkamers),2 badkamers,"Ligbad, 2 douches en 2 toiletten",3 woonlagen,Schuifpui,A,HR-glas en volledig geïsoleerd,Cv-ketel en gedeeltelijke vloerverwarming,Cv-ketel en elektrische boiler,"Nefit (gas gestookt combiketel uit 2017, eigen...",,Zonneterras,Dakterras aanwezig en frans balkon aanwezig,Parkeerkelder en parkeerplaats,NaN,NaN,Betaald parkeren,"Aan rustige weg, aan vaarwater, aan water, in ...",NaN,Gelegen op het zuidwesten,€ 14 per maand,Gedeeltelijk gestoffeerd,"33 m² (9,4 meter diep en 3,51 meter breed)",Inpandig,Elektra en stromend water,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Bosplaat 25,1025 AR,Amsterdam,€ 700.000 kosten koper,€ 5.833,19 september 2020,Beschikbaar,In overleg,"Bungalow, tussenwoning (semi-bungalow)",Bestaande bouw,1973,Plat dak,,160 m²,273 m³,4 kamers (3 slaapkamers),

In [5]:
df.columns

Index(['address', 'postcode', 'city', 'OVE-Vraagprijs',
       'OVE-Vraagprijs per m²', 'OVE-Aangeboden sinds', 'OVE-Status',
       'OVE-Aanvaarding', 'BOU-Soort woonhuis', 'BOU-Soort bouw',
       'BOU-Bouwjaar', 'BOU-Soort dak', 'OPP-', 'OPP-Perceel', 'OPP-Inhoud',
       'IND-Aantal kamers', 'IND-Aantal badkamers',
       'IND-Badkamervoorzieningen', 'IND-Aantal woonlagen',
       'IND-Voorzieningen', 'ENE-Energielabel', 'ENE-Isolatie',
       'ENE-Verwarming', 'ENE-Warm water', 'ENE-Cv-ketel', 'KAD-', 'BUI-Tuin',
       'BUI-Balkon/dakterras', 'GAR-Soort garage', 'GAR-Capaciteit',
       'GAR-Voorzieningen', 'PAR-Soort parkeergelegenheid', 'BUI-Ligging',
       'BUI-Voortuin', 'BUI-Ligging tuin', 'OVE-Servicekosten',
       'BOU-Specifiek', 'BUI-Zonneterras', 'BER-Schuur/berging',
       'BER-Voorzieningen', 'BUI-Achtertuin', 'BER-Isolatie', 'BOU-Keurmerken',
       'BUI-Patio/atrium', 'OVE-Bijdrage VvE', 'BOU-Soort appartement',
       'BOU-Bouwperiode', 'IND-Gelegen op', 'VVE-In

In [6]:
df.isna().sum()

address                              0
postcode                             0
city                                 0
OVE-Vraagprijs                       3
OVE-Vraagprijs per m²              119
OVE-Aangeboden sinds                 1
OVE-Status                           1
OVE-Aanvaarding                      6
BOU-Soort woonhuis                3451
BOU-Soort bouw                       5
BOU-Bouwjaar                       369
BOU-Soort dak                     1728
OPP-                               113
OPP-Perceel                       3531
OPP-Inhoud                         113
IND-Aantal kamers                  113
IND-Aantal badkamers               517
IND-Badkamervoorzieningen          913
IND-Aantal woonlagen               113
IND-Voorzieningen                 1186
ENE-Energielabel                  1005
ENE-Isolatie                      1025
ENE-Verwarming                     472
ENE-Warm water                     597
ENE-Cv-ketel                      1628
KAD-                     

In [7]:
df.dtypes

address                            object
postcode                           object
city                               object
OVE-Vraagprijs                     object
OVE-Vraagprijs per m²              object
OVE-Aangeboden sinds               object
OVE-Status                         object
OVE-Aanvaarding                    object
BOU-Soort woonhuis                 object
BOU-Soort bouw                     object
BOU-Bouwjaar                       object
BOU-Soort dak                      object
OPP-                               object
OPP-Perceel                        object
OPP-Inhoud                         object
IND-Aantal kamers                  object
IND-Aantal badkamers               object
IND-Badkamervoorzieningen          object
IND-Aantal woonlagen               object
IND-Voorzieningen                  object
ENE-Energielabel                   object
ENE-Isolatie                       object
ENE-Verwarming                     object
ENE-Warm water                    

# Make it happen

In [42]:
# Order of categories
col_trans = ["OVE", "BOU", "OPP",
             "IND", "ENE", "BUI",
             "GAR", "BER", "PAR", 
             "VVE", "KAD", "BED",
             "VEI"]

order = [x
         for i, col in enumerate(col_trans)
         for x in df.columns
         if x.startswith(col_trans[i])] 

df = df[["address", "postcode", "city"] + order]

In [43]:
# Rename columns
cols = {'OVE-Vraagprijs': 'asking_price',
        'OVE-Vraagprijs per m²': 'price_m2',
        'OVE-Aangeboden sinds': 'days_online',
        'OVE-Status': 'status',
        'OVE-Aanvaarding': 'acceptance',
        'BOU-Soort woonhuis': 'property_type',
        'BOU-Soort bouw': 'new_build',
        'BOU-Bouwjaar': 'build_year',
        'BOU-Soort dak': 'roof_type',
        'OPP-': 'opp',
        'OPP-Perceel': 'property_m2',
        'OPP-Inhoud': 'property_m3',
        'IND-Aantal kamers': 'num_rooms',
        'IND-Aantal badkamers': 'bathrooms',
        'IND-Badkamervoorzieningen': 'bathroom_features',
        'IND-Aantal woonlagen': 'floors',
        'IND-Voorzieningen':  'features',
        'ENE-Energielabel': 'energy_label',
        'ENE-Isolatie': 'isolation',
        'ENE-Verwarming': 'heating',
        'ENE-Warm water': 'hot_water',
        'ENE-Cv-ketel': 'boiler',
        'KAD-': 'kadaster',
        'BUI-Tuin': 'garden',
        'BUI-Balkon/dakterras': 'balcony',
        'GAR-Soort garage': 'garage_type',
        'GAR-Capaciteit': 'garage_size',
        'GAR-Voorzieningen': 'garage_features',
        'PAR-Soort parkeergelegenheid': 'parking',
        'BUI-Ligging': 'environment',
        'BUI-Voortuin': 'garden_front',
        'BUI-Ligging tuin': 'garden_orientation',
        'OVE-Servicekosten': 'service_fees_pm',
        'BOU-Specifiek': 'specials',
        'BUI-Zonneterras': 'terrace',
        'BER-Schuur/berging': 'storage_type',
        'BER-Voorzieningen': 'storage_features',
        'BUI-Achtertuin': 'garden_back',
        'BER-Isolatie': 'storage_isolation',
        'BOU-Keurmerken': 'certificates',
        'BUI-Patio/atrium': 'garden_patio',
        'OVE-Bijdrage VvE': 'vve_contribution',
        'BOU-Soort appartement': 'apartment_type',
        'BOU-Bouwperiode': 'build_era',
        'IND-Gelegen op': 'apartment_level',
        'VVE-Inschrijving KvK': 'vve_kvk',
        'VVE-Jaarlijkse vergadering': 'vve_am',
        'VVE-Periodieke bijdrage': 'vve_per_contr',
        'VVE-Reservefonds aanwezig': 'vve_reserve_fund',
        'VVE-Onderhoudsplan': 'vve_maintenance',
        'VVE-Opstalverzekering': 'vve_insurance',
        'GAR-Isolatie': 'garage_isolation',
        'BOU-Toegankelijkheid': 'accessibility',
        'OVE-Oorspronkelijke vraagprijs': 'asking_price_original',
        'ENE-Voorlopig energielabel': 'energy_label_temp',
        'OVE-Huurprijs': 'rent_price',
        'OVE-Huurovereenkomst': 'rental_agreement',
        'BUI-Plaats': 'garden_plaats',
        'BUI-Zijtuin': 'garden_side',
        'OVE-Oorspronkelijke huurprijs': 'rent_price_original',
        'BOU-Soort overig aanbod': 'prop_extra_type',
        'BED-Praktijkruimte': 'comp_practice',
        'OVE-Koopmengvorm': 'sale_type',
        'BOU-Soort parkeergelegenheid': 'parking_type',
        'IND-Capaciteit': 'parking_capacity',
        'IND-Afmetingen': 'prop_extra_dimensions',
        'VEI-Prijs': 'auction_price',
        'VEI-Veilingperiode': 'auction_period',
        'VEI-Soort veiling': 'auction_type',
        'VEI-Veilingpartij': 'auction_party',
        'BED-Bedrijfsruimte': 'company_space',
        'BED-Kantoorruimte': 'office_space',
        'BED-Winkelruimte': 'store_space',
        'IND-Perceel': 'ground_area',
        'BOU-Soort object': 'prop_build_area'}
df.rename(cols, axis=1, inplace=True)

In [44]:
# Initial drop of columns with little meaning
drop = ['status', 'acceptance', 'asking_price_original', 'rent_price',
        'rental_agreement', 'rent_price_original', 'sale_type', 'certificates',
        'accessibility', 'prop_extra_type', 'parking_type', 'prop_build_area', 
        'opp', 'parking_capacity', 'prop_extra_dimensions', 'ground_area', 
        'garden_plaats', 'garden_side', 'garage_size', 'garage_features',
        'garage_isolation', 'kadaster', 'comp_practice', 'company_space', 
        'office_space', 'store_space', 'auction_price', 'auction_period', 
        'auction_type', 'auction_party']
df.drop(columns=drop, inplace=True)

In [45]:
# Drop rows without asking price
df.dropna(subset=["asking_price"], inplace=True)

In [46]:
# Drop listings that are just garages and such
df = (df.drop(df[df["apartment_type"].isna() 
           & df["property_type"].isna()].index)
      .reset_index(drop=True))

In [47]:
# Drop new build projects which aren't specific
df.drop(df[df["address"]
           .str.contains(r"bouw|appartemnt|wonen", 
                                      case=False, 
                                      regex=True)]
        .index, 
        inplace=True)

In [14]:
# Convert columns in euro to numeric
euro = ["asking_price", "vve_contribution", 
        "service_fees_pm", "price_m2"]

for e in euro:
    df[e] = extract_num(df[e], "price")

In [15]:
# Calculate days since posting
df["days_online"] = (df["days_online"]
                     .apply(convert_elapsed_time))

In [16]:
# Convert to binary
# Current yes/no columns
binary = ["vve_kvk", "vve_am", "vve_reserve_fund", 
          "vve_maintenance", "vve_insurance"]

# Fill NaN with 0
df[["vve_per_contr"] + binary] = (df[["vve_per_contr"] + binary].fillna(0))

# Straight forward columns
for col in binary:
    df[col] = np.where(df[col] == "Ja", 1, 0)

# Columns containing numeric values as well
df["vve_per_contr"] = np.where(df["vve_per_contr"]
                               .str.contains("ja", case=False), 
                               1, 0)

# Other binary oppositions
df["new_build"] = np.where(df["new_build"] == "Nieuwbouw", 
                           1, 0)

In [17]:
# Calculate mean of build period
df["build_era"] = (df["build_era"]
                   .apply(build_era)
                   .astype(int, errors="ignore"))

In [18]:
# Use mean of build period if build year is null
df["build_year"] = np.where(df["build_year"].notnull(), 
                            df["build_year"], 
                            df["build_era"])
df.drop(columns=["build_era"], inplace=True)

In [19]:
# Extract number from build year
df["build_year"] = extract_num(df["build_year"], "year")

In [20]:
# Extract numbers from area/volume
for m in ["property_m2", "property_m3"]:
    df[m] = extract_num(df[m], "meter")

In [21]:
# Make tags of property and apartment types into dummy columns (prefix pt)
listing_type(df)

In [22]:
# Convert roof type string into 'type' and 'form' dummy columns (prefix rt and rf)
df = (pd.concat([df, roof_description(df["roof_type"].copy())], axis=1)
      .drop(columns=["roof_type"]))

In [23]:
# Extract number of rooms and bedrooms
df["rooms"] = extract_num(df["num_rooms"], "rooms")
df["bedrooms"] = extract_num(df["num_rooms"], "bedrooms")

# Where no bedrooms mentioned, use rooms-1
df.loc[(df["rooms"] > 1) & (df["bedrooms"] == 0), "bedrooms"] = df["rooms"] - 1
# Fix where rooms & bedrooms stated both 2
df.loc[(df["rooms"] == 2) & (df["bedrooms"] == 2), "rooms"] = 3
# Otherwise for equal number use bedrooms-1 (if not already 1 bedroom)
df.loc[(df["rooms"] == df["bedrooms"]) 
       & (df["rooms"] != 1), "bedrooms"] = df["bedrooms"] - 1

# Drop original rooms column
df.drop(columns="num_rooms", inplace=True)

# TODO

In [1]:
# Make dummies for extra floors
pats = {"attic": "zolder|vliering", "cellar": "kelder"}
for key, pat in pats.items():
    df[key] = create_dummy(df["floors"], pat)

# Number of floors in house
df["floors"] = np.where(not df["bovenwoning"] & not df["benedenwoning"], extract_num(df["floors"], "floors"), 0)

# Level of apartment
df["apartment_level"] = extract_num(df["apartment_floor"], "app_level")


SyntaxError: invalid syntax (<ipython-input-1-049ec9103a05>, line 5)

In [25]:
df[df["num_floors"] > 10]

,address,postcode,city,asking_price,price_m2,days_online,service_fees_pm,vve_contribution,new_build,build_year,specials,property_m2,property_m3,bathrooms,bathroom_features,floors,features,apartment_floor,energy_label,isolation,heating,hot_water,boiler,energy_label_temp,garden,balcony,environment,garden_front,garden_orientation,terrace,garden_back,garden_patio,garage_type,storage_type,storage_features,storage_isolation,parking,vve_kvk,vve_am,vve_per_contr,vve_reserve_fund,vve_maintenance,vve_insurance,pt_villa,pt_herenhuis,pt_penthouse,pt_2-onder-1-kapwoning,pt_bungalow,pt_hofjeswoning,pt_landhuis,pt_waterwoning,pt_eengezinswoning,pt_corridorflat,pt_tussenwoning,pt_hoekwoning,pt_benedenwoning,pt_appartement,pt_drive-in_woning,pt_portiekwoning,rf_zadeldak,rf_plat dak,rf_lessenaardak,rf_mansarde,rf_samengesteld,rf_tentdak,rf_dwarskap,rf_schilddak,rt_bitumin,rt_pannen,rt_kunststof,rt_leisteen,rt_metaal,rt_asbest,rt_riet,rooms,bedrooms,attic,cellar,num_floors
68,Jan Evertsenstraat 617,1061 XZ,Amsterdam,400000,4494,12,112,112.0,0,2008,NaN,0,292,1 badkamer en 1 apart toilet,Douche,19 woonlagen,Mechanische ventilatie,12e woonlaag,A,Volledig geïsoleerd,Stadsverwarming,Stadsverwarming,NaN,NaN,NaN,Balkon aanwezig,"Aan park, aan rustige weg, in woonwijk, open l...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Betaald parkeren, parkeergarage en parkeerverg...",1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,3,2,0,0,19
345,Oeverpad 200,1068 PH,Amsterdam,485000,3911,14,165,0.0,0,2007,NaN,0,385,1 badkamer en 1 apart toilet,Ligbad en toilet,12 woonlagen,Lift en mechanische ventilatie,11e woonlaag,A,Volledig geïsoleerd,Cv-ketel,Cv-ketel,2007 ( combiketel),NaN,NaN,NaN,"Aan park, in woonwijk, open ligging en vrij ui...",NaN,NaN,NaN,NaN,NaN,Parkeerplaats,Inpandig,Elektra en stromend water,NaN,Op eigen terrein,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,3,2,0,0,12
458,Gustav Mahlerlaan 78,1082 ME,Amsterdam,775000,6624,56,422,0.0,0,2008,NaN,0,411,1 badkamer en 1 apart toilet,Sauna en ligbad,24 woonlagen,"Alarminstallatie, balansventilatie, , jacuzzi,...",3e woonlaag,A,Volledig geïsoleerd,Stadsverwarming,Stadsverwarming,NaN,NaN,NaN,NaN,"Aan rustige weg, beschutte ligging, in centrum...",NaN,NaN,NaN,NaN,NaN,NaN,Inpandig,NaN,NaN,Betaald parkeren en parkeergarage,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,3,2,0,0,24
635,Backershagen 29,1082 GR,Amsterdam,550000,4231,42,224,224.0,0,1965,NaN,0,403,1 badkamer en 1 apart toilet,Ligbad en douche,12 woonlagen,Lift,6e woonlaag,D,Dubbel glas,Blokverwarming,Centrale voorziening,NaN,NaN,NaN,NaN,"Aan park, beschutte ligging, in bosrijke omgev...",NaN,NaN,NaN,NaN,NaN,NaN,Box,Elektra,NaN,"Betaald parkeren, openbaar parkeren en parkeer...",1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,4,3,0,0,12
1460,Langswater 220,1069 TS,Amsterdam,225000,3082,42,200,0.0,0,1966,NaN,0,235,1 badkamer en 1 apart toilet,Douche,12 woonlagen,Lift en mechanische ventilatie,10e woonlaag,F,Volledig geïsoleerd,Blokverwarming,Elektrische boiler,NaN,NaN,NaN,Balkon aanwezig,"In woonwijk, open ligging en vrij uitzicht",NaN,NaN,NaN,NaN,NaN,NaN,Box,NaN,Geen isolatie,Openbaar parkeren,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,4,3,0,0,12
1471,Tommaso Albinonistraat 62,1083 HM,Amsterdam,625000,6313,120,0,0.0,0,2006,NaN,0,270,1 badkamer en 1 apart toilet,"Ligbad, douche en toilet",12 woonlagen,"Lift, mechanische ventilatie en TV kabel",5e woonlaag,A,Volledig geïsoleerd,Stadsverwarming,Stadsverwarming,NaN,NaN,NaN,Balkon aanwezig,Vrij uitzicht,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Betaald parkeren,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,3,2,0,0,12
1512,Langswater 9,1069 TR,Amsterdam,225000,3125,15,208,0.0,0,1965,NaN,0,230,1 badkamer en 1 apart toilet,Douche,12 woonlagen,Lift,1e woonlaag,NaN,Volledig geïsoleerd,Blokverwarming,Cv-ketel,NaN,E,NaN,Balkon aanwezig,In woonwijk en vrij uitzicht

## Geolocation

In [26]:
key = getpass()

········


In [27]:
# Load neighborhoods of Amsterdam
gdf = gpd.read_file("../data/ams_neighborhoods.shp")

In [28]:
gdf.head()

,Buurt_code,Buurt,Buurtcombi,Stadsdeel_,Opp_m2,FID,geometry
0,F81d,Calandlaan/Lelylaan,F81,F,275360.0,1,"POLYGON ((4.80081 52.35517, 4.80906 52.35684, ..."
1,F81e,Osdorp Zuidoost,F81,F,519366.0,2,"POLYGON ((4.81859 52.35751, 4.81863 52.35629, ..."
2,F82a,Osdorp Midden Noord,F82,F,215541.0,3,"POLYGON ((4.78666 52.36271, 4.79533 52.36443, ..."
3,F82b,Osdorp Midden Zuid,F82,F,258379.0,4,"POLYGON ((4.78830 52.35973, 4.79692 52.36147, ..."
4,F82c,Zuidwestkwadrant Osdorp Noord,F82,F,240774.0,5,"POLYGON ((4.79021 52.35620, 4.79926 52.35802, ..."


In [29]:
# Make the full address into one column
df = (df.assign(full_address=df[["address", "postcode", "city"]]
                .agg(", ".join, axis=1))
      .drop(columns=["address", "postcode", "city"]))

In [30]:
df.insert(0, column="address", value=df.pop("full_address"))
df.head()

,address,asking_price,price_m2,days_online,service_fees_pm,vve_contribution,new_build,build_year,specials,property_m2,property_m3,bathrooms,bathroom_features,floors,features,apartment_floor,energy_label,isolation,heating,hot_water,boiler,energy_label_temp,garden,balcony,environment,garden_front,garden_orientation,terrace,garden_back,garden_patio,garage_type,storage_type,storage_features,storage_isolation,parking,vve_kvk,vve_am,vve_per_contr,vve_reserve_fund,vve_maintenance,vve_insurance,pt_villa,pt_herenhuis,pt_penthouse,pt_2-onder-1-kapwoning,pt_bungalow,pt_hofjeswoning,pt_landhuis,pt_waterwoning,pt_eengezinswoning,pt_corridorflat,pt_tussenwoning,pt_hoekwoning,pt_benedenwoning,pt_appartement,pt_drive-in_woning,pt_portiekwoning,rf_zadeldak,rf_plat dak,rf_lessenaardak,rf_mansarde,rf_samengesteld,rf_tentdak,rf_dwarskap,rf_schilddak,rt_bitumin,rt_pannen,rt_kunststof,rt_leisteen,rt_metaal,rt_asbest,rt_riet,rooms,bedrooms,attic,cellar,num_floors
0,"Buiksloterbreek 41, 1034 XC, Amsterdam",475000,5220,5,0,0.0,0,1994,NaN,51,379,1 badkamer en 1 apart toilet,Douche en toilet,2 woonlagen,"Buitenzonwering, schuifpui en TV kabel",NaN,B,Volledig geïsoleerd,Cv-ketel,Cv-ketel,Intergas HRE (gas gestookt combiketel uit 2010...,NaN,Zonneterras,Dakterras aanwezig,NaN,NaN,NaN,NaN,NaN,NaN,Inpandig,NaN,NaN,NaN,"Betaald parkeren, openbaar parkeren en parkeer...",0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,4,3,0,0,2
1,"Arènpalmstraat 16, 1104 DB, Amsterdam",425000,3400,14,0,0.0,0,2008,NaN,98,429,1 badkamer en 1 apart toilet,"Ligbad, douche en toilet",3 woonlagen,Mechanische ventilatie en TV kabel,NaN,A,Volledig geïsoleerd,Cv-ketel en gedeeltelijke vloerverwarming,Cv-ketel,Gas gestookt combiketel,NaN,Voortuin,Dakterras aanwezig en balkon aanwezig,"Aan rustige weg, aan water, in woonwijk en vri...","36 m² (7 meter diep en 5,09 meter breed)",Gelegen op het zuidoosten,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Op eigen terrein en openbaar parkeren,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,6,4,0,0,3
2,"Brigantijnkade 46, 1086 VB, Amsterdam",625000,4340,13,14,0.0,0,2009,Gedeeltelijk gestoffeerd,133,519,2 badkamers,"Ligbad, 2 douches en 2 toiletten",3 woonlagen,Schuifpui,NaN,A,HR-glas en volledig geïsoleerd,Cv-ketel en gedeeltelijke vloerverwarming,Cv-ketel en elektrische boiler,"Nefit (gas gestookt combiketel uit 2017, eigen...",NaN,Zonneterras,Dakterras aanwezig en frans balkon aanwezig,"Aan rustige weg, aan vaarwater, aan water, in ...",NaN,Gelegen op het zuidwesten,"33 m² (9,4 meter diep en 3,51 meter breed)",NaN,NaN,Parkeerkelder en parkeerplaats,Inpandig,Elektra en stromend water,NaN,Betaald parkeren,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,5,4,0,0,3
3,"Bosplaat 25, 1025 AR, Amsterdam",700000,5833,12,0,0.0,0,1973,NaN,160,273,1 badkamer en 1 apart toilet,Toilet,2 woonlagen,Dakraam en schuifpui,NaN,C,Dubbel glas,Cv-ketel,Elektrische boiler,"Atag (gas gestookt combiketel uit 2010, eigendom)",NaN,Achtertuin en voortuin,Balkon aanwezig,Aan rustige weg en in woonwijk,NaN,Gelegen op het zuidwesten bereikbaar via achterom,NaN,"104 m² (16,71 meter diep en 6,21 meter breed)",NaN,NaN,Aangebouwde stenen berging,Elektra,NaN,"Betaald parkeren, openbaar parkeren en parkeer...",0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,4,3,0,0,2
4,"Ben van Meerendonkstraat 156, 1087 LN, Amsterdam",779000,4839,13,43,0.0,0,2010,Gedeeltelijk gestoffeerd,142,580,2 badkamers en 1 apart toilet,"Ligbad, 2 douches en 2 toiletten",3 woonlagen,Schuifpui en TV kabel,NaN,A,Dubbel glas en volledig geïsoleerd,Stadsverwarming en gehele vloerverwarming,Stadsverwarming,NaN,NaN,Achtertuin,NaN,"Aan rustige weg, aan vaarwater, aan water, in ...",NaN,Gelegen op het zuidwesten,NaN,"78 m² (13,53 meter diep en 5,74 meter breed)",NaN,Parkeerkelder en parkeerplaats,Box,NaN,NaN,Betaald parkeren,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,5,4,0,0,3


In [31]:
# Get the coordinates of every location
c = df.iloc[:5].copy()
coords = (gpd.tools.geocode(c["address"], 
                            provider="google", 
                            api_key=key))

In [32]:
# Make sure downloaded coordinates are in correct format
coords.to_crs(epsg=4326, inplace=True)

# Add the neighborhood for every location
geo_combi = (gpd.sjoin(coords, 
                       gdf[["Buurt", "geometry"]], 
                       how="inner", 
                       op='intersects') 
             .drop(columns=["geometry", "index_right"]))
geo_combi.head()

,address,Buurt
0,"Buiksloterbreek 41, 1034 XC Amsterdam, Netherl...",Buiksloterbreek
1,"Arènpalmstraat 16, 1103 SE Amsterdam, Netherlands",Grunder/Koningshoef
2,"Brigantijnkade 46, 1086 XZ Amsterdam, Netherlands",Steigereiland Noord
3,"Bosplaat 25, 1025 AR Amsterdam, Netherlands",Buikslotermeer Noord
4,"Ben van Meerendonkstraat 156, 1087 NA Amsterda...",Haveneiland Oost


In [33]:
# Merge with original dataframe
c = (c.merge(geo_combi, 
             left_index=True, 
             right_index=True)
     .drop(columns=["address_y"]))

## Bathrooms

In [40]:
df = (df.assign(num_bathrooms=extract_num(df["bathrooms"], "bathrooms"), 
                num_toilets=extract_num(df["bathrooms"], "toilets"))
      .drop(columns="bathrooms"))